In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# spark = SparkSession.builder \
#     .appName("Kafka to Minio") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Kafka to Minio") \
    .config("spark.task.maxFailures", "1") \
    .config("spark.hadoop.fs.s3a.endpoint", "minio:9000") \
    .config("spark.hadoop.s3a.access.key", "wFw2nWmmNHNivuqh") \
    .config("spark.hadoop.fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.ssl.enabled", False) \
    .getOrCreate()
# tiAoMHTLqpoi7TKMVXQGPR7N0VT6dRsJ
# E0QTNG3vCGYwqpar

spark.sparkContext.setLogLevel('INFO')
# spark.sparkContext.setLogLevel('ERROR')

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker:9092") \
    .option("subscribe", "user") \
    .load()

df = df.selectExpr("CAST(value AS STRING)")

query = df \
    .writeStream \
    .format("json") \
    .option("path", "s3a://raw/user") \
    .option("checkpointLocation", "/tmp/checkpoints") \
    .trigger(processingTime='1 seconds') \
    .start()

query.awaitTermination()